In [3]:
import tabula
import pandas as pd
import numpy as np
import math

## STEPWISE PROCESS
- EDIT THE PDF TO REMOVE UNWANTED IMAGES AND HEADINGS
- CONVERT THE PDF TO CSV
- IMPORT THE CSV FILE
- INITIAL CLEANING AND CONVERTING OF NUMERIC COLUMNS
- *Remove header rows


### FINDING REOCCURING PAYMENTS (TOKENIZATION - research it)
- Separate debit and credit by putting them into new columns
- check average monthly income and expenses
- check descriptions to check spending habits
- check if the person is a salary earner (reoccuring payment every month)


### STEPS TO MERGING THE DETAILS COLUMN
- Get the iloc(index) of the row with valid date
- add the details of the row before and after the row with valid date to the valid date row details



In [4]:
# file_path = 'Statement.pdf'
# df = tabula.read_pdf(file_path, pages='all')

In [5]:
# convert PDF into CSV file
# tabula.convert_into("Statement.pdf", "Statement.csv", output_format="csv", pages='all')

In [6]:
statement_df = pd.read_csv('Statement.csv')

In [7]:
statement_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328 entries, 0 to 327
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Date                 276 non-null    object
 1   Transaction Details  310 non-null    object
 2   Refrence             270 non-null    object
 3   Value Date           276 non-null    object
 4   withdrawals          275 non-null    object
 5   Lodgements           275 non-null    object
 6   Balance              275 non-null    object
dtypes: object(7)
memory usage: 18.1+ KB


In [16]:
statement_df.head()

,Date,Transaction Details,Refrence,Value Date,Withdrawals,Lodgements,Balance,Credits,Debits
0,17-AUG-2021,SURV. GAIUS W.K. ASSOR/ONB TRF FROM SURV. GAIU...,000007210817112721616155327395,17-AUG-21,0.00,3000.0,3046.22,1,0
1,17-AUG-2021,TRF/Self/FRM CHIMMENE DAMIETE GAIUS- ASSOR TO ...,000014210817140146296069460523,17-AUG-21,1010.75,0.0,2035.47,0,1
2,19-AUG-2021,TRF FRM GAIUS ASSOR\]. Trip to BABCOCK IFO CHI...,DBL000822135620210818896,18-AUG-21,0.00,20000.0,22035.47,1,0
3,19-AUG-2021,TRF/Self/FRM CHIMMENE DAMIETE GAIUS- ASSOR TO ...,000014210819161141270501074247,19-AUG-21,10026.88,0.0,2008.59,0,1
4,20-AUG-2021,GAIUS-ASSOR ARUEBUBECHIM BUBIGAIUS- ASSOR ARUE...,000013210821000057000286834329,21-AUG-21,0.00,3000.0,5008.59,1,0


### CLEANING PROCESS

In [9]:
#Drop header rows and reset index
statement_df.drop(statement_df.index[statement_df['Date'] == 'Date'], inplace=True)
statement_df.reset_index(drop=True, inplace=True)

#rename withdrawals to start with uppercase
statement_df.rename(columns={'withdrawals': 'Withdrawals'}, inplace=True)

In [10]:
#format numeric string colums removing spaces and comma
cols = ['Withdrawals', 'Lodgements', 'Balance']
statement_df[cols] = statement_df[cols].apply(lambda x: x.str.strip())
statement_df[cols] = statement_df[cols].apply(lambda x: x.str.replace(',',''))

#convert number columns to numeric
statement_df[cols] = statement_df[cols].apply(pd.to_numeric, errors='coerce')

In [11]:
#Get the index for rows with valid date
valid_idx = list(statement_df.index[statement_df['Date'].isna()==False])

# valid_idx = [val for val in valid_idx if val < 121]
valid_idx.remove(0) #remove header row

In [12]:
#combine split rows together
for val in valid_idx:
    if type(statement_df.loc[val-1,'Date']) == float and type(statement_df.loc[val+1,'Date']) == float:
        #for rows split into 2
        if type(statement_df.loc[val,'Transaction Details']) == float:
            statement_df.loc[val,'Transaction Details'] = statement_df.loc[val-1,'Transaction Details'] + ' ' + statement_df.loc[val+1,'Transaction Details']
        #for rows split into 3
        else:
            statement_df.loc[val,'Transaction Details'] = statement_df.loc[val-1,'Transaction Details'] + ' ' + statement_df.loc[val,'Transaction Details'] + ' ' + statement_df.loc[val+1,'Transaction Details']


In [13]:
#Drop unwanted columns (previously split columns)
statement_df = statement_df.dropna()

#reset index
statement_df.reset_index(drop=True, inplace=True)


In [14]:
def credits(row):
    if row['Lodgements'] != 0:
        val = 1
    else:
        val = 0
    return val

def debits(row):
    if row['Withdrawals'] != 0:
        val = 1
    else:
        val = 0
    return val

In [15]:
#separate debit and credit transanctions
statement_df['Credits'] = statement_df.apply(credits, axis=1)

statement_df['Debits'] = statement_df.apply(debits, axis=1)

In [17]:
statement_df.to_csv('Cleaned_Statement.csv', index=False)

## Analysis 

In [18]:
statement = pd.read_csv('Cleaned_Statement.csv')

In [19]:
statement.head()

,Date,Transaction Details,Refrence,Value Date,Withdrawals,Lodgements,Balance,Credits,Debits
0,17-AUG-2021,SURV. GAIUS W.K. ASSOR/ONB TRF FROM SURV. GAIU...,000007210817112721616155327395,17-AUG-21,0.00,3000.0,3046.22,1,0
1,17-AUG-2021,TRF/Self/FRM CHIMMENE DAMIETE GAIUS- ASSOR TO ...,000014210817140146296069460523,17-AUG-21,1010.75,0.0,2035.47,0,1
2,19-AUG-2021,TRF FRM GAIUS ASSOR\]. Trip to BABCOCK IFO CHI...,DBL000822135620210818896,18-AUG-21,0.00,20000.0,22035.47,1,0
3,19-AUG-2021,TRF/Self/FRM CHIMMENE DAMIETE GAIUS- ASSOR TO ...,000014210819161141270501074247,19-AUG-21,10026.88,0.0,2008.59,0,1
4,20-AUG-2021,GAIUS-ASSOR ARUEBUBECHIM BUBIGAIUS- ASSOR ARUE...,000013210821000057000286834329,21-AUG-21,0.00,3000.0,5008.59,1,0


[]